In [1]:
import os
os.chdir("..")
from src.dataset import VietnameseTextDataset, prepare_vietnamese_dataset, load_texts_from_folder
from tokenizers import Tokenizer
import torch
from glob import glob
import torch.nn as nn

Device set to use cuda:0


In [2]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Punctuation, Sequence, Digits, Metaspace
from tokenizers.normalizers import NFC, NFD, Sequence as NormalizerSequence
from tokenizers.processors import TemplateProcessing
import os
import glob
from src.tokenizer import VietnameseTokenizer

## 1. Preprocessing
**Segment Word**

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("NlpHUST/vi-word-segmentation")
model = AutoModelForTokenClassification.from_pretrained("NlpHUST/vi-word-segmentation")

nlp = pipeline("token-classification", model=model, tokenizer=tokenizer)

In [ ]:
example = """thơ lục bát: 
ai ơi xa bến quê hương
nhớ về quê mẹ nắng sương đây nè
nhớ sao những buổi trưa hè
võng đưa cót két gió hè hiu hiu
lời ru của mẹ dấu yêu
ngọt ngào êm dịu mẹ yêu con nhiều
lời mẹ năm tháng sớm chiều
con nghe nhớ mãi những điều ru ta
lòng mẹ ôi thật bao la
thái bình rộng lớn thương là mênh mông
một đời lưng mẹ đã còng
vai mang tay xách lo chồng chăm con
mong sao con lớn nên người
thân mẹ có cực vẫn cười thương con
vì đời mẹ sống cho con
gom bao mệt nhọc mãi còn vòng tay
mong chờ cho đến một ngày
công thành danh toại là ngày mẹ mong
dù nay mẹ đã xa rồi
con đây nhớ mãi mẹ ôi vạn lần
nguyện rằng ghi nhớ công ân
sinh thành dưỡng dục mẫu thân đời đời"""

def word_segment(text: str) -> str:
    ner_results = nlp(text)
    example_tok = ""
    for e in ner_results:
        if "##" in e["word"]:
            example_tok = example_tok + e["word"].replace("##","")
        elif e["entity"] =="I":
            example_tok = example_tok + "_" + e["word"]
        else:
            example_tok = example_tok + " " + e["word"]
    return example_tok

In [8]:
input_filename = 'train_data_1/vietnam_poetry.txt'   # your input file
output_filename = 'train_data_1/vietnam_poetry_3.txt' # desired output file

In [4]:
with open(input_filename, "r", encoding="utf-8") as f:
    data = f.read()

new_data = data.split("<s>")

In [5]:
len(new_data)

171189

In [7]:
def save_poetry_data(data, output_file, max_words=512):
    with open(output_file, 'w', encoding='utf-8') as f:
        for stanza in data:  # Mỗi stanza là một phần tử trong danh sách
            if not isinstance(stanza, str) or not stanza.strip():
                continue  # Bỏ qua nếu không phải chuỗi hoặc rỗng

            words = stanza.strip().split()
            
            # Chia thành các chunk ≤ 512 từ
            for i in range(0, len(words), max_words):
                chunk = ' '.join(words[i:i + max_words])
                f.write(f"<s> {chunk}\n")
    
    print(f"Đã lưu {len(data)} khổ thơ (đã chia nhỏ) vào {output_file}")

In [9]:
save_poetry_data(new_data, output_filename, max_words=400)

Đã lưu 171189 khổ thơ (đã chia nhỏ) vào train_data_1/vietnam_poetry_3.txt


In [ ]:
with open(output_filename, "w", encoding="utf-8") as outfile:
    for line in new_data[1:]:
        print(line)
        text = word_segment(line)
        outfile.write("<s>" + text + "\n")

In [ ]:
vn_tokenizer = VietnameseTokenizer()

In [ ]:
# Build tokenizer with Vietnamese-specific settings
trainer = vn_tokenizer.build_tokenizer(
    vocab_size=25000,
    min_frequency=2
)

# Get training files
train_files = glob.glob(os.path.join("./train_data", "*.txt"))

if not train_files:
    print("No training files found in ./train_data/")
    print("Please add Vietnamese text files (.txt) to ./train_data/ directory")
    exit()

print(f"Found {len(train_files)} training files")

# Train the tokenizer
print("Training tokenizer...")
vn_tokenizer.train(train_files, trainer)

# Setup post-processor
vn_tokenizer.setup_post_processor()

# Save tokenizer
vn_tokenizer.save("vietnamese_enhanced_tokenizer.json")
print("Tokenizer saved as 'vietnamese_enhanced_tokenizer.json'")

# Test the tokenizer
vn_tokenizer.test_tokenizer()

In [ ]:
import tiktoken
def count_tokens(string: str, model_name: str = "gpt-4o") -> int:
    """Returns the number of tokens in a text string for a specific model."""
    encoding = tiktoken.encoding_for_model(model_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [ ]:
with open('./train_data/vietnam_poetry.txt', 'r', encoding='utf-8') as f:
    data = f.read()

In [ ]:
print(count_tokens(data))

In [ ]:


# 1. KHỞI TẠO TOKENIZER CHUẨN
tokenizer = Tokenizer(BPE(
    unk_token="[UNK]",
    fuse_unk=True  # Tự động gộp các UNK hiếm
))

# 2. NORMALIZER QUAN TRỌNG NHẤT (FIX DẤU THANH)
tokenizer.normalizer = NormalizerSequence([
    NFC(),  # ⚡️ CHÌA KHÓA VÀNG: Giữ nguyên dấu thanh (Á = 1 ký tự, KHÔNG tách A + dấu)
    NFD(),  # Tối ưu cho 1 số trường hợp hiếm
])

# 3. PRE-TOKENIZER CHUYÊN BIỆT CHO TIẾNG VIỆT
tokenizer.pre_tokenizer = Sequence([
    Digits(individual_digits=True),  # Tách số riêng (123 → ["1","2","3"])
    Punctuation(),                   # Tách DẤU CÂU đúng cách (.,?!;:→ riêng)
    Metaspace(replacement="▁")  # Dùng▁ thay dấu cách (phù hợp BPE)
])

# 4. TRAINER VỚI THAM SỐ TỐI ƯU
trainer = BpeTrainer(
    vocab_size=22000,          # Tăng nhẹ từ 20K → 22K để đủ chỗ cho dấu câu
    min_frequency=3,           # Chỉ lấy token xuất hiện ≥ 3 lần
    special_tokens=[
        "[PAD]", "[UNK]", 
        "[EOS]", "[BOS]",
        "[MASK]"               # Cần cho 1 số kỹ thuật augment
    ],
    # initial_alphabet=["▁"],    # Ký tự bắt đầu từ (Metaspace)
    show_progress=True
)

# 5. TRAINING (DÙNG TẤT CẢ FILE .txt)
files = glob.glob(os.path.join("./train_data", "*.txt"))
tokenizer.train(files, trainer)

# 6. POST-PROCESSOR (QUAN TRỌNG CHO GENERATION)
tokenizer.post_processor = TemplateProcessing(
    single="[BOS] $A [EOS]",
    pair="[BOS] $A [EOS] $B:1 [EOS]:1",
    special_tokens=[
        ("[BOS]", tokenizer.token_to_id("[BOS]")),
        ("[EOS]", tokenizer.token_to_id("[EOS]"))
    ]
)

# 7. LƯU VÀ KIỂM TRA
tokenizer.save("vietnamese_pro_tokenizer.json")

In [ ]:
# tokenizer = Tokenizer.from_file("vietnamese_pro_tokenizer.json")
tokenizer = vn_tokenizer.load("vietnamese_enhanced_tokenizer.json")

In [ ]:
text = """bướm xòe đôi cánh lả lơi
tìm hoa hút mật tuyệt vời thơm ngon
nắng chiều mơn trớn mạ non
hoàng hôn tắt lịm trăng tròn tắm sông
xuân về hạ tới thu đông
xong mưa trời tặng cầu vồng trên cao
mía dừa múi mít ngọt ngào
chanh cam bí mướp mận đào xoài nho"""
output = tokenizer.encode(text)

print("Tokens:", output.tokens)
print("IDs:", output.ids)

In [ ]:
output_text = tokenizer.decode(output.ids, skip_special_tokens=True)
print("Decoded Text:", output_text)

In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer_hf = PreTrainedTokenizerFast(
    tokenizer_file="vietnamese_bytelevel_tokenizer.json",
    bos_token="[BOS]",
    eos_token="[EOS]",
    pad_token="[PAD]",
    unk_token="[UNK]"
)

In [ ]:
def count_tokens(text):
    encoding = tokenizer.encode(text)
    return len(encoding.tokens) # trả về số lượng và danh sách token (tuỳ chọn)

# Ví dụ
text = "Xin chào, mình là sinh viên trường đại học Khoa học Tự nhiên."

num_tokens = count_tokens(data)
print(f"Số lượng token: {num_tokens}")

In [ ]:
files = glob(os.path.join("./data", "*.txt"))
files

In [ ]:
sentences = []

for data_file in files:
    with open(data_file, 'r', encoding='utf-8') as f:
        raw_text = f.read()
        sentences.append(raw_text)
# sentences[]

In [ ]:
from src.tokenizer import VietnamesePreprocessor
import random
preprocessor = VietnamesePreprocessor()

In [ ]:
raw_texts = load_texts_from_folder("train_data_1")
all_sentences = []
for text in raw_texts:
    cleaned_text = preprocessor.clean_text(text)
    sentences_from_file = preprocessor.segment_sentences(cleaned_text)
    all_sentences.extend(sentences_from_file) # Use extend to add all sentences to one list

train_split = 0.8
random.shuffle(all_sentences)
split_idx = int(len(all_sentences) * train_split)
train_sentences = all_sentences[:split_idx]
val_sentences = all_sentences[split_idx:]

In [ ]:
train_sentences[:3]

In [ ]:
train_data = VietnameseTextDataset(texts=train_sentences, tokenizer=tokenizer, max_length=256, stride=128)
train_data.__getitem__(1)

In [ ]:
data = VietnameseTextDataset(texts=sentences, tokenizer=tokenizer, max_length=256, stride=128)

In [ ]:
data.__getitem__(0)

In [ ]:
raw_text = load_texts_from_folder("data")

In [ ]:
train_loader, val_loader = prepare_vietnamese_dataset(data_folder="train_data_1", tokenizer=tokenizer)

In [ ]:
# Test the data loader
for i, batch in enumerate(train_loader):
    if i >= 2:  # Only show first 2 batches
        break
        
    print(f"\nBatch {i + 1}:")
    print(f"  Input IDs shape: {batch['input_ids'].shape}")
    print(f"  Target IDs shape: {batch['target_ids'].shape}")
    print(f"  Attention mask shape: {batch['attention_mask'].shape}")
    
    # Show first sequence in batch
    input_seq = batch['input_ids'][2]
    target_seq = batch['target_ids'][2]
    
    print(f"  Sample input:  {input_seq[:50].tolist()}...")
    print(f"  Sample target: {target_seq[:50].tolist()}...")
    
    # Decode sample
    decoded_input = tokenizer.decode(input_seq.tolist())
    decoded_target = tokenizer.decode(target_seq.tolist())
    
    print(f"  Decoded input:  '{decoded_input[:50]}...'")
    print(f"  Decoded target: '{decoded_target[:50]}...'")

In [ ]:
config = {
    # Data configuration
    'data_folder': 'data1',
    'tokenizer_file': 'vietnamese_bpe_tokenizer.json',
    'vocab_size': 25000,
    'max_seq_len': 512,
    'train_split': 0.8,
    
    # Model configuration
    'd_model': 768,
    'n_heads': 12,
    'n_layers': 12,
    'd_ff': 3072,
    'dropout': 0.1,
    
    # Training configuration
    'batch_size': 16,
    'learning_rate': 3e-5,
    'weight_decay': 0.01,
    'num_epochs': 20,
    'warmup_steps': 5000,
    'device': 'auto',  # 'cuda', 'cpu', or 'auto'
    
    # Generation configuration
    'temperature': 0.8,
    'top_k': 10,
    'top_p': 0.9,
    'max_new_tokens': 100,
    
    # Save configuration
    'model_save_path': 'vietnamese_transformer_best.pt',
    'config_save_path': 'training_config.json'
}

In [ ]:
from src.model import VietnameseTransformer

model = VietnameseTransformer(
    vocab_size=tokenizer.get_vocab_size(),
    d_model=config['d_model'],
    n_heads=config['n_heads'],
    n_layers=config['n_layers'],
    d_ff=config['d_ff'],
    max_seq_len=config['max_seq_len'],
    dropout=config['dropout'],
    pad_token_id=tokenizer.token_to_id("[PAD]")
)
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

model.to("cuda")
print(f"✅ Model created successfully!")
print(f"   Total parameters: {total_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")
print(f"   Model size: {total_params * 4 / 1024 / 1024:.2f} MB (float32)")

In [ ]:
from src.trainer import VietnameseTrainer
from src.helpers import test_generation
# Step 3: Initialize trainer
print(f"\n{'='*20} STEP 3: TRAINING SETUP {'='*20}")

trainer = VietnameseTrainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    tokenizer=tokenizer,
    lr=config['learning_rate'],
    weight_decay=config['weight_decay'],
    warmup_steps=config['warmup_steps'],
    device=config['device']
)

print(f"✅ Trainer initialized!")
print(f"   Device: {trainer.device}")
print(f"   Learning rate: {config['learning_rate']}")
print(f"   Batch size: {config['batch_size']}")

# Test initial generation (before training)
print(f"\n{'='*20} INITIAL GENERATION TEST {'='*20}")
print("Testing generation before training (should be random):")
test_generation(model, tokenizer, trainer.device, ["thơ lục bát: ai ơi xa bến quê hương"])

In [ ]:
# Step 4: Train the model
print(f"\n{'='*20} STEP 4: TRAINING {'='*20}")
print(f"Starting training for {config['num_epochs']} epochs...")
print("Press Ctrl+C to stop training early\n")

try:
    trainer.train(
        num_epochs=config['num_epochs'],
        save_path=config['model_save_path']
    )
    
    print(f"\n🎉 Training completed successfully!")
    
except KeyboardInterrupt:
    print(f"\n⏹️  Training interrupted by user")
    print("Saving current model state...")
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': trainer.optimizer.state_dict(),
        'train_losses': trainer.train_losses,
        'val_losses': trainer.val_losses,
        # 'tokenizer': tokenizer
    }, 'vietnamese_transformer_interrupted.pt')
    print("Model saved as 'vietnamese_transformer_interrupted.pt'")

In [ ]:
# Step 6: Final generation test
print(f"\n{'='*20} STEP 6: FINAL GENERATION TEST {'='*20}")
print(config['model_save_path'])
# Load best model for testing
if os.path.exists(config['model_save_path']):
    # Load the entire checkpoint dictionary without the `weights_only` flag
    checkpoint = torch.load(config['model_save_path'], map_location=trainer.device, weights_only=True)

    # Load only the model's weights from the loaded dictionary
    model.load_state_dict(checkpoint['model_state_dict'])
    print("✅ Loaded best model for testing")

In [ ]:
def test_generation(model, tokenizer, device, test_cases=None):
    """Test text generation with various examples"""
    if test_cases is None:
        test_cases = [
            "Truyện Kiều là",
        ]
    
    print("\n" + "="*60)
    print("🎯 TESTING TEXT GENERATION")
    print("="*60)
    
    model.eval()
    
    for i, prompt in enumerate(test_cases, 1):
        print(f"\n--- Test {i} ---")
        print(f"Input: '{prompt}'")
        
        # Encode input
        input_ids = torch.tensor(
            [tokenizer.encode(prompt, add_special_tokens=False).ids],
            device=device
        )
        
        # Generate with different settings
        generation_configs = [
            {'temperature': 0.7, 'top_k': 50, 'top_p': 0.9, 'max_new_tokens': 15, 'name': 'Balanced'},
            {'temperature': 1.0, 'top_k': 20, 'top_p': 0.8, 'max_new_tokens': 15, 'name': 'Creative'},
            {'temperature': 0.0, 'top_k': 5, 'top_p': 1.0, 'max_new_tokens': 15, 'name': 'Conservative'}
        ]
        
        for config in generation_configs:
            with torch.no_grad():
                generated = model.generate(
                    input_ids,
                    temperature=config['temperature'],
                    top_k=config['top_k'],
                    top_p=config['top_p'],
                    max_new_tokens=config['max_new_tokens'],
                    do_sample=True
                )
            
            generated_text = tokenizer.decode(generated[0].cpu().tolist())
            print(f"  {config['name']}: '{generated_text}'")

In [ ]:
test_generation(model, tokenizer, device="cuda", test_cases = ["Sóng được sáng tác bởi"])